<a href="https://colab.research.google.com/github/sandovalmedeiros/PythonGeo/blob/main/Semana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

In [21]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 22 15:28:33 2022

@author: sarvi0
"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler


#baseline 76.555%
#modelo 3 76.794%

In [22]:
#%% abrir o datase de treino e teste

train = pd.read_csv('/content/drive/MyDrive/ELT579/Semana 1/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ELT579/Semana 1/test.csv')

In [ ]:
#%% pre-processamento dos dados

#descrição estátistica das features núméricas
est = train.describe()

print(train.info())

#verificar valores nulos ou NAN
print(train.isnull().sum())

print(test.isnull().sum())

#mapear as colunas
col = pd.Series(list(train.columns))

X_train = train.drop(['PassengerId', 'Survived'], axis = 1)

X_test = test.drop(['PassengerId'], axis = 1)

In [24]:
#criar feature

def criar_features(X):
  subs = {'female':1, 'male':0}
  X['mulher'] = X['Sex'].replace(subs)

  X['Fare'] = X['Fare'].fillna(X['Fare'].mean())

  X['Age'] = X['Age'].fillna(X['Age'].mean())

  X['Embarked'] = X['Embarked'].fillna('S')

  subs = {'S':1, 'C':2, 'Q':3}
  X['porto'] = X['Embarked'].replace(subs)

  X['crianca'] = 1
  X['crianca'] = np.where(X['Age'] < 12, 1, 0)

  return X

X_train = criar_features(X_train)
X_test = criar_features(X_test)

In [25]:
#%% Selecionar as features

features = ['Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'mulher', 'porto', 'crianca']

X_train = X_train[features]
X_test = X_test[features]

y_train = train['Survived']

In [ ]:
#%% Visualização

import matplotlib.pyplot as plt

for i in X_train.columns:
    plt.hist(X_train[i])
    plt.title(i)
    plt.show()

In [27]:
#%% Groupy

gp = train.groupby(['Survived']).count()

#%% pivot_table

table = pd.pivot_table(train, index = ['Survived'], columns = ['Pclass'], values = 'PassengerId', aggfunc = 'count')

In [28]:
#%% Padronização das variáveis

scaler = StandardScaler() #media 0 e desvio padrão 1

X_train_sc = scaler.fit_transform(X_train)

X_test_sc = scaler.transform(X_test)


In [29]:
#%% modelo e validação cruzada

#Logistic Regression
model_lr = LogisticRegression (random_state= 0 )

score = cross_val_score(model_lr, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8058302122347065


In [30]:
#%% Naive Bayes para Classificação

from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()

score = cross_val_score(model_nb, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8036079900124845


In [31]:
#%% KNN para classificação
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors= 5, p = 2)

score = cross_val_score(model_knn, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.811498127340824


In [32]:
#%% SVM para classificação
from sklearn.svm import SVC

model_svc = SVC(C = 3, kernel = 'rbf', degree = 2, gamma = 0.1)

score = cross_val_score(model_svc, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8316479400749064


In [33]:
#%% Decision Tree

from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, min_samples_split = 2, min_samples_leaf = 1, random_state = 0)

score = cross_val_score(model_dt, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8170536828963796


In [34]:
#%% Random Forest

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(criterion = 'entropy', n_estimators = 100, max_depth = 5, min_samples_split = 2, min_samples_leaf = 1, random_state = 0)

score = cross_val_score(model_rf, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8261048689138576


In [35]:
#%% Otimização de hiperparametros

from skopt import gp_minimize

def treinar_modelo(parametros):

  model_rf = RandomForestClassifier(criterion = parametros[0], n_estimators = parametros[1], max_depth = parametros[2],
                                    min_samples_split = parametros[3], min_samples_leaf = parametros[4], random_state = 0, n_jobs = -1 )

  score = cross_val_score(model_rf, X_train_sc, y_train, cv = 10)

  mean_score = np.mean(score)

  print(np.mean(score))

  return -mean_score

parametros = [('entropy', 'gini'),
              (100, 1000),
              (3, 20),
              (2, 10),
              (1, 10)]


otimos = gp_minimize(treinar_modelo, parametros, random_state = 0, verbose = 1, n_calls = 30, n_random_starts = 10  )


print(otimos.fun, otimos.x)

Iteration No: 1 started. Evaluating function at random point.
0.8227465667915107
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 42.8722
Function value obtained: -0.8227
Current minimum: -0.8227
Iteration No: 2 started. Evaluating function at random point.
0.829450686641698
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 11.1675
Function value obtained: -0.8295
Current minimum: -0.8295
Iteration No: 3 started. Evaluating function at random point.
0.8249937578027466
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 24.0109
Function value obtained: -0.8250
Current minimum: -0.8295
Iteration No: 4 started. Evaluating function at random point.
0.8227340823970037
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 22.0498
Function value obtained: -0.8227
Current minimum: -0.8295
Iteration No: 5 started. Evaluating function at random point.
0.8272409488139825
Iteration No: 5 ended. Evaluation done at random point.
Time

In [36]:
#%% Ensanble model (Voting)
from sklearn.ensemble import VotingClassifier

model_voting = VotingClassifier(estimators = [('LR', model_lr), ('KNN', model_knn), ('SVC', model_svc), ('RF', model_rf)], voting = 'hard')

model_voting.fit(X_train_sc, y_train)

score = cross_val_score(model_voting, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8294257178526842


In [37]:
#%% modelo final

model_rf = RandomForestClassifier(criterion = otimos.x[0], n_estimators = otimos.x[1], max_depth = otimos.x[2],
                                    min_samples_split = otimos.x[3], min_samples_leaf = otimos.x[4], random_state = 0, n_jobs = -1 )

model_rf.fit(X_train_sc, y_train)

y_pred = model_rf.predict(X_train_sc)

mc = confusion_matrix(y_train, y_pred)
print(mc)

score = model_rf.score(X_train_sc, y_train)
print(score)


[[521  28]
 [114 228]]
0.8406285072951739


In [51]:
#%% predição nos dados de teste

y_pred = model_voting.predict(X_test_sc)

submission = pd.DataFrame(test['PassengerId'])

submission['Survived'] = y_pred

submission.to_csv('submission_professor2.csv', index = False)


In [41]:
print(submission)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [40]:
print(y_pred)

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [39]:
print(test)

     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  
0      male  34.5      0      0 

In [50]:
print(X_test_sc)

[[ 0.82737724  0.36944878 -0.4745452  ... -0.73769513  2.57919938
  -0.28744485]
 [ 0.82737724  1.33137817  0.43279337 ...  1.35557354 -0.56883712
  -0.28744485]
 [-0.36936484  2.48569343 -0.4745452  ... -0.73769513  2.57919938
  -0.28744485]
 ...
 [ 0.82737724  0.67726619 -0.4745452  ... -0.73769513 -0.56883712
  -0.28744485]
 [ 0.82737724  0.04413122 -0.4745452  ... -0.73769513 -0.56883712
  -0.28744485]
 [ 0.82737724  0.04413122  0.43279337 ... -0.73769513  1.00518113
  -0.28744485]]
